# Fast low-rank nonnegative tensor factorizations in tensor train format

In [1]:
%reload_ext autoreload
%autoreload 2

import sys
sys.path.append('./src')

import numpy as np
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt
import scipy.io as spio
from time import time

from tt import TT

In [2]:
## TTSVD, GetFullTensor

sizes = [10, 20, 20]
ranks = [1, 10, 10, 1]

tt = TT(sizes=sizes, ranks=ranks, seed=42)
full = TT.GetFullTensor(tt.GetCores())
np.linalg.norm(full - TT.GetFullTensor(TT.TTSVD(full, [1, 10, 10, 1]))) / np.linalg.norm(full)

8.952125031870687e-15

In [3]:
## DotProduct

tt1 = TT(sizes=sizes, ranks=ranks)
full1 = TT.GetFullTensor(tt1.GetCores())

tt2 = TT(sizes=sizes, ranks=ranks)
full2 = TT.GetFullTensor(tt2.GetCores())

print(TT.DotProduct(tt1, tt2) - (full1 * full2).sum())

0.0


In [4]:
## Hadamard product

tt1 = TT(sizes=sizes, ranks=ranks)
full1 = TT.GetFullTensor(tt1.GetCores())

tt2 = TT(sizes=sizes, ranks=ranks)
full2 = TT.GetFullTensor(tt2.GetCores())

print(np.linalg.norm(full1 * full2 - TT.GetFullTensor((tt1 * tt2).GetCores())))

4.1032412651917504e-12


In [5]:
## Orthogonalize

tt1 = TT(sizes=[5, 7, 8, 10], ranks=[1, 3, 3, 3, 1])
tt1.Orthogonalize()

t = tt1._cores[0].squeeze(0)
print(np.linalg.norm(t.T @ t - np.eye(t.shape[1])))
for p in range(1, len(tt1._cores) - 1):
    t = t @ tt1._cores[p].reshape(tt1._ranks[p], -1)
    t = t.reshape(t.shape[0] * tt1._sizes[p], tt1._ranks[p + 1])
    
    print(np.linalg.norm(t.T @ t - np.eye(t.shape[1])))

4.802744877930169e-16
3.215879113371732e-16
9.497529839230413e-16


In [6]:
## Compress

tt1 = TT(sizes=[5, 7, 8, 10], ranks=[1, 3, 3, 3, 1])
tt1.Compress(1e-6)

t = tt1._cores[3].squeeze(2)
print(np.linalg.norm(t @ t.T - np.eye(t.shape[0])))
for p in range(len(tt1._cores) - 2, 0, -1):
    t = tt1._cores[p].reshape(-1, tt1._ranks[p + 1]) @ t
    t = t.reshape(tt1._ranks[p], -1)
    
    print(np.linalg.norm(t @ t.T - np.eye(t.shape[0])))

6.070726916179205e-16
2.020622328642659e-15
2.122280863703296e-15
